<a href="https://colab.research.google.com/github/wolffg777/ait_datascience/blob/main/MusicRec01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO:
WORD2VEC KERAS CLASSIFICATION PROBLEM
- first clean up
- testing it to see improvement
- implementing primary_genres and secondary_genres as inputs
- implement output as album-artist name pair
- implementing dates (as continuous)
- easier things first
- spotify data integration


-brainstorm an idea for the name of the independent study on askbanner (limited to 30 characters)




check - train on domain text
notcheck - pretrained w2v
notcheck - use pretrained with domain adaptation  



In [2]:
import gensim.downloader as api
path = api.load("word2vec-google-news-300", return_path=True)
print(path)

/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [3]:
from google.colab import files
files.upload()
!pip install kaggle
!chmod 600 /root/.kaggle/kaggle.json
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!kaggle datasets download -d tobennao/rym-top-5000
!unzip rym-top-5000.zip
!pip install spotipy

Saving kaggle.json to kaggle.json
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
  0% 0.00/863k [00:00<?, ?B/s]
100% 863k/863k [00:00<00:00, 128MB/s]
Archive:  rym-top-5000.zip
  inflating: rym_clean1.csv          
  inflating: rym_raw1.csv            
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 8.3 MB/s eta 0:00:00


In [4]:
#imports
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer


# Load the word2vec model
model = KeyedVectors.load_word2vec_format(path, binary=True)

In [14]:
#Load data
df = pd.read_csv('rym_clean1.csv')

In [15]:
# Remove unnecessary columns
df = df.drop(columns=['avg_rating'])
df = df.drop(columns=['rating_count'])
df = df.drop(columns=['review_count'])
df = df.drop(columns=['Unnamed: 0'])
df = df.drop(columns=['position'])
df = df.drop(columns=['release_type'])
df = df.drop(columns=['artist_name'])
df = df.drop(columns=['release_date'])
df.head(5)

,release_name,primary_genres,secondary_genres,descriptors
0,OK Computer,"Alternative Rock, Art Rock",NaN,"melancholic, anxious, futuristic, malevocals, ..."
1,Kid A,"Art Rock, Experimental Rock, Electronic","Ambient, Electronic, IDM","cold, melancholic, futuristic, anxious, atmosp..."
2,The Dark Side of the Moon,"Art Rock, Progressive Rock","Psychedelic Rock, Space Rock","philosophical, atmospheric, introspective, exi..."
3,Loveless,"Shoegaze, Noise Pop","Dream Pop, Neo-Psychedelia","noisy, ethereal, atmospheric, romantic, love, ..."
4,My Beautiful Dark Twisted Fantasy,"Pop Rap, Hip Hop",Art Pop,"epic, boastful, passionate, sampling, hedonist..."


In [16]:
#Reformat descriptors properly
df['descriptors'] = df['descriptors'].str.split(', ')
desc_words = df['descriptors']

In [17]:
#Tokenize descriptors
texts = df['descriptors'].tolist()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [18]:
#Vectorize descriptors
def descriptors_to_vector(descriptors):
    vectors = [model[word] for word in descriptors if word in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Apply the function to the dataframe
df['vector'] = df['descriptors'].apply(descriptors_to_vector)

In [19]:
# print(df['vector'].iloc[0])
# print(type(df['vector']))

In [20]:
padded_descriptors = pad_sequences(df['vector'].tolist(), maxlen=100, dtype='float32')
# padded_descriptors = np.stack(df['vector'].values)
# type(padded_descriptors)
# print(len(padded_descriptors[0]))
# df.head(5)

In [21]:
embedding_dim = 300  # As the Word2Vec model you downloaded has 300 dimensions
vocab_size = len(tokenizer.word_index) + 1

# Create an embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():  # tokenizer should be the tokenizer you used to process your text
    if word in model:
        embedding_vector = model[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector


In [22]:
encoder = LabelEncoder()
df['release_name_label'] = encoder.fit_transform(df['release_name'])

# # Prepare training data
X = np.array(padded_descriptors, dtype='float32')
# X = np.array(df['vector'])
y = df['release_name_label'].values

In [23]:
# print(type(X))


In [24]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

modelSeq = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=padded_descriptors.shape[1],
              weights=[embedding_matrix], trainable=False),  # Using pre-trained embeddings and setting them non-trainable
    LSTM(128, return_sequences=True),  # Return sequences if you want to stack LSTMs
    Dropout(0.2),  # Add dropout for regularization
    LSTM(64),  # Another LSTM layer
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(len(df['release_name'].unique()), activation='softmax')
])

modelSeq.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)

# Assuming you've created a validation set called X_val and y_val
history = modelSeq.fit(X, y, epochs=25, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stop, model_checkpoint])


Epoch 1/25
79/79 [==============================] - ETA: 0s - loss: 8.5188 - accuracy: 0.0000e+00
Epoch 1: val_loss improved from inf to 8.50420, saving model to best_model.h5
79/79 [==============================] - 25s 267ms/step - loss: 8.5188 - accuracy: 0.0000e+00 - val_loss: 8.5042 - val_accuracy: 0.0010
Epoch 2/25
79/79 [==============================] - ETA: 0s - loss: 8.5080 - accuracy: 2.0000e-04
Epoch 2: val_loss improved from 8.50420 to 8.50420, saving model to best_model.h5
79/79 [==============================] - 21s 263ms/step - loss: 8.5080 - accuracy: 2.0000e-04 - val_loss: 8.5042 - val_accuracy: 0.0000e+00
Epoch 3/25
79/79 [==============================] - ETA: 0s - loss: 8.5079 - accuracy: 8.0000e-04
Epoch 3: val_loss did not improve from 8.50420
79/79 [==============================] - 21s 271ms/step - loss: 8.5079 - accuracy: 8.0000e-04 - val_loss: 8.5042 - val_accuracy: 0.0000e+00
Epoch 4/25
79/79 [==============================] - ETA: 0s - loss: 8.5078 - accura

things to do: stricter filtering of descriptors
one-hot the genres, word2vec (number of dimensions depends on embedding model, basic word2vec is 30 - 100), the descriptors and average the value and put it in space. tutorials on youtube seem good.

try to find a taxonomy of genres online, if not split up words and vaguely separating the musical words

Partial credit accuracy/reward system where "radiohead" for atmospheric even if the album recommended is not atmospheric

In [26]:
import os
import base64
from requests import post, get
import json
import spotipy
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials


os.environ['SPOTIPY_CLIENT_ID'] = '08a0640020ff4e5d9713aaeacfbd1886'
os.environ['SPOTIPY_CLIENT_SECRET'] = '81cefe24f94b4f24af9fe1ad2e70ef4a'
os.environ['SPOTIPY_REDIRECT_URI'] = 'http://example.com/callback'

#Authorization Code Flow
scopes = "user-library-read user-top-read user-follow-read"

sp = spotipy.Spotify(
        auth_manager=spotipy.SpotifyOAuth(
          scope=scopes, open_browser=False))

In [30]:
#Get Specified User's Playlists
playlists = sp.user_playlists('frankthetank922')

while playlists:
    for i, playlist in enumerate(playlists['items']):
        #print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
        print("%4d %s" % (i + 1 + playlists['offset'], playlist['name']))

    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None


   1 My Playlist #5
   2 PRE-WORKOUT PLAYLIST 
   3 fire
   4 Deja Vu (Initial D)
   5 chill


In [31]:
album_name = "OK Computer"
artist_name = "Radiohead"

# Search for the album
results = sp.search(q=f"album:{album_name} artist:{artist_name}", type="album")

album_id = results['albums']['items'][0]['id']

album_tracks = sp.album_tracks(album_id)

for track in album_tracks['items']:
    print(track['name'])


Airbag
Paranoid Android
Subterranean Homesick Alien
Exit Music (For A Film)
Let Down
Karma Police
Fitter Happier
Electioneering
Climbing Up the Walls
No Surprises
Lucky
The Tourist


model architecture: pytorch simple neural network (feedforward neural network).

get a model working before break, even if its simpler (in terms of model architecture and pre-processing).

in recommendation stage, top n choices of different artists because otherwise it will just recommend songs from the same album
|
also figure out a way to vectorize/umbrella categories for genres
|
add column for frequency of each genre like
col 1 : space rock
col 2 : rock
col 3 : frequency 3
|
also separate the words in the descriptors (ie "malevocals" should be "male vocals")

still do this, but only useful for bringing in new information that is song specific, like lyrics and popularity

In [32]:
# CODE GRAVEYARD

# # Flatten the lists in the "descriptors" column
# flattened_descriptors = df['descriptors'].explode()

# # Count the number of unique strings
# unique_count = flattened_descriptors.nunique()

# flattened_descriptors.head(5)

# # Use value_counts to count the frequency of elements in the column
# element_counts = flattened_descriptors.value_counts()

# # Print the result
# print(element_counts)



# # One-hot-encoding for DESCRIPTORS

# # Turn descriptors into a list of strings
# df['descriptors'] = df['descriptors'].str.split(',')

# descriptors_encoded = df['descriptors'].str.join('|').str.get_dummies()

# descriptor_frequencies = descriptors_encoded.iloc[:, 2:].sum()

# # print (descriptor_frequencies)

# rarity_threshold = 10

# # Clean column names by stripping spaces
# descriptors_encoded.columns = descriptors_encoded.columns.str.strip()

# rare_descriptors = descriptor_frequencies[descriptor_frequencies < rarity_threshold]

# # print(rare_descriptors)
# # print(len(rare_descriptors))